In [5]:
# Cell 1: Setup and Imports - Using testing_utils
import sys
import time
import numpy as np
import warnings
from IPython.display import clear_output

clear_output(wait=True)
warnings.filterwarnings('ignore')

# Import testing utilities - provides all shared infrastructure
from qectostim.testing import (
    # Status indicators
    STATUS_OK, STATUS_WARN, STATUS_SKIP, STATUS_FAIL,
    # Module management
    clear_qectostim_modules,
    # Test function
    test_decoder_on_code,
    # Decoder/code loading utilities
    load_all_decoders,
    discover_and_categorize_codes,
    print_code_summary,
    # Results formatting
    format_status,
    compute_summary_stats,
)

# Clear module cache for fresh testing
clear_qectostim_modules()

# Discover and categorize all codes
# NOTE: Bosonic (GKP, rotor), qudit (Galois), and fracton (XCube, Haah) codes
# are excluded by default as they don't work with standard qubit decoders
categories, all_codes = discover_and_categorize_codes(
    max_qubits=100,
    include_qldpc=True,
    include_subsystem=True,
    include_floquet=True,
    include_bosonic=False,   # Exclude - continuous variables
    include_qudit=False,     # Exclude - d>2 dimensions
    include_fracton=False,   # Exclude - exotic excitations
)

# Print summary
print_code_summary(categories, "DECODER SMOKE TEST - ALL CODE TYPES")

DECODER SMOKE TEST - ALL CODE TYPES

Total discovered: 58 codes
  CSS: 35
  Color: 4
  Non-CSS: 5
  Subsystem: 4
  Floquet: 1
  QLDPC: 9

CSS Codes:
Code Name                                   n   k   d
-------------------------------------------------------
  FourQubit422_[[4,2,2]]                    4   2   2
  C6                                        6   2   2
  Steane_713                                7   1   3
  Shor_91                                   9   1   3
  ReedMuller_15_1_3                        15   1   3
  Hamming_CSS_7                             7   1   3
  Code_832                                  8   3   2
  Repetition_3                              3   1   3
  Repetition_5                              5   1   5
  Repetition_7                              7   1   7
  RotatedSurface_[[9,1,3]]                  9   1   3
  RotatedSurface_[[25,1,5]]                25   1   5
  ToricCode_3x3                            18   2   3
  ToricCode_5x5                        

In [ ]:
# Debug cell: Check QLDPC factory loading
from qectostim.codes import qldpc as qldpc_module

print("Checking QLDPC factories in qldpc module:")
qldpc_factories = ['HGPHamming7', 'BBGrossCode', 'HDX_4', 'HDX_6', 'QuantumTanner_4', 'DLV_8',
                   'ExpanderLP_10_3', 'DHLV_5_1', 'BalancedProductRep5']

for name in qldpc_factories:
    factory = getattr(qldpc_module, name, None)
    if factory is None:
        print(f"  ❌ {name}: NOT FOUND in qldpc module")
    else:
        try:
            code = factory()
            print(f"  ✅ {name}: Success - {type(code).__name__} with n={getattr(code, 'n', '?')}")
        except Exception as e:
            print(f"  ⚠️ {name}: Found but failed - {e}")

# Use the actual patterns from discovery.py
print("\nChecking if QLDPC patterns would match:")
qldpc_names = ['Hypergraph', 'Bicycle', 'Lifted', 'Fiber', 'HGP', 'BB', 'GB',
               'HDX', 'Expander', 'DHLV', 'Campbell', 'Tanner', 'DLV', 
               'Lossless', 'HigherDim', 'Balanced', 'QLDPC']
for name in ['HDXCode', 'ExpanderLPCode', 'DHLVCode', 'QuantumTannerCode', 'DinurLinVidickCode']:
    matches = any(pat in name for pat in qldpc_names)
    print(f"  {name}: {'✅' if matches else '❌'} matches QLDPC pattern")

Checking 4D Toric codes in surface module:
  ⚠️ ToricCode4D: Found but failed - ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 96 is different from 64)
  ⚠️ ToricCode4D_2: Found but failed - ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 96 is different from 64)
  ⚠️ ToricCode4D_3: Found but failed - ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 486 is different from 324)
  ⚠️ LoopToricCode4D: Found but failed - TypeError: TopologicalCSSCode4D.__init__() got an unexpected keyword argument 'hx'
  ⚠️ LoopToric4D_2: Found but failed - TypeError: TopologicalCSSCode4D.__init__() got an unexpected keyword argument 'hx'
  ⚠️ ToricCode3D: Found but failed - TypeError: ToricCode3D.__init__() got an unexpected keyword argument 'Lx'
  ⚠️ ToricCode3DFaces: Foun

In [6]:
# Cell 2: Load Available Decoders - Using testing_utils

decoder_classes = load_all_decoders()
print(f"Loaded {len(decoder_classes)} decoders: {list(decoder_classes.keys())}")

Loaded 9 decoders: ['PyMatching', 'FusionBlossom', 'BeliefMatching', 'BPOSD', 'Tesseract', 'UnionFind', 'MLE', 'Hypergraph', 'Chromobius']


In [7]:
# Cell 3: Test Helper Function - Now imported from testing_utils
# The test_decoder_on_code function is already imported from qectostim.testing
# It handles:
#   - Color codes using ColorCodeMemoryExperiment for Chromobius-compatible DEMs
#   - CSS codes using CSSMemoryExperiment
#   - Other codes using StabilizerMemoryExperiment
#   - Chromobius will SKIP on non-color codes (expected behavior)

print("✓ test_decoder_on_code imported from qectostim.testing")
print("  - Color codes use ColorCodeMemoryExperiment (Chromobius-compatible DEMs)")
print("  - Other codes use CSSMemoryExperiment or StabilizerMemoryExperiment")
print("  - Chromobius will SKIP on non-color codes (expected)")

✓ test_decoder_on_code imported from qectostim.testing
  - Color codes use ColorCodeMemoryExperiment (Chromobius-compatible DEMs)
  - Other codes use CSSMemoryExperiment or StabilizerMemoryExperiment
  - Chromobius will SKIP on non-color codes (expected)


In [8]:
# Cell 4: DECODER × CODE COMPATIBILITY MATRIX - ALL CODES
from IPython.display import clear_output
clear_output(wait=True)

p = 0.01
shots = 1000

# Store results - reinitialize to be safe
full_results = {}

# Build output lines to print at end (prevents output duplication)
output_lines = []
output_lines.append("="*130)
output_lines.append("DECODER × CODE COMPATIBILITY MATRIX - ALL CODE TYPES")
output_lines.append("="*130)
output_lines.append(f"\nTesting {len(all_codes)} decoder/code combinations at p={p}, {shots} shots...")
output_lines.append("Code types: CSS, Non-CSS, Subsystem, Floquet, QLDPC, Color")
output_lines.append("Note: Chromobius requires color-code-like DEMs (shows SKIP for other codes)\n")

# Header
dec_names = list(decoder_classes.keys())
header = f"{'Code':<35} {'Type':<10}"
for dec_name in dec_names:
    header += f" | {dec_name[:10]:^12}"
output_lines.append(header)
output_lines.append("-" * len(header))

# Process each code ONCE
for code_name, (code_type, code) in all_codes.items():
    row = f"{code_name:<35} {code_type:<10}"
    full_results[code_name] = {'type': code_type}
    
    for dec_name in dec_names:
        # test_decoder_on_code returns TestResult object
        result = test_decoder_on_code(
            code=code,
            decoder_class=decoder_classes[dec_name],
            decoder_name=dec_name,
            code_type=code_type,
            p=p,
            shots=shots
        )
        # Convert to dict for storage
        result_dict = result.to_dict()
        full_results[code_name][dec_name] = result_dict
        
        if result.status == 'OK':
            ler = result.ler
            if result.warnings:
                cell = f"⚠️{ler:.4f}" if ler is not None else "⚠️N/A"
            else:
                cell = f"✓ {ler:.4f}" if ler is not None else "✓ N/A"    
        elif result.status == 'WARN':
            cell = f"⚠️WARN"
        elif result.status == 'SKIP':
            cell = f"- SKIP"
        else:
            cell = f"✗ FAIL"
        
        row += f" | {cell:^12}"
    
    output_lines.append(row)

output_lines.append("-" * len(header))
output_lines.append(f"\nLegend: ✓=pass, ⚠️=warning, -=skip (expected), ✗=fail")
output_lines.append(f"Total codes tested: {len(full_results)}")

# Single print statement
print("\n".join(output_lines))

DECODER × CODE COMPATIBILITY MATRIX - ALL CODE TYPES

Testing 58 decoder/code combinations at p=0.01, 1000 shots...
Code types: CSS, Non-CSS, Subsystem, Floquet, QLDPC, Color
Note: Chromobius requires color-code-like DEMs (shows SKIP for other codes)

Code                                Type       |  PyMatching  |  FusionBlos  |  BeliefMatc  |    BPOSD     |  Tesseract   |  UnionFind   |     MLE      |  Hypergraph  |  Chromobius 
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
FourQubit422_[[4,2,2]]              CSS        |   ✓ 0.0520   |   ✓ 0.0490   |   ✓ 0.0550   |   ✓ 0.0510   |   ✓ 0.0370   |   ✓ 0.0490   |   ✓ 0.0460   |   ✓ 0.0590   |    - SKIP   
C6                                  CSS        |   ✓ 0.0880   |   ✓ 0.1020   |   ✓ 0.1050   |   ✓ 0.1030   |   ✓ 0.0890   |   ✓ 0.0810   |   ✓ 0.0630   |   ✓ 0.0730   |    - SKIP   
Steane_713          

In [6]:
# Cell 5: LER COMPARISON TABLE - ALL CODES
from IPython.display import clear_output
clear_output(wait=True)

print("="*140)
print("LER COMPARISON TABLE (p=0.01) - ALL CODE TYPES")
print("="*140)
print(f"\nTotal codes in all_codes: {len(all_codes)}")
print(f"Total codes in full_results: {len(full_results)}")
print("\nLower is better. Best decoder for each code highlighted.")
print("SKIP = decoder incompatible (e.g., Chromobius requires color-code DEMs)\n")

# Header
dec_names = list(decoder_classes.keys())
header = f"{'Code':<35} | {'Type':<10} | {'d':>2} | {'No-decode':>10}"
for dec_name in dec_names:
    header += f" | {dec_name[:10]:>10}"
header += " | Best"
print(header)
print("-" * len(header))

# Use full_results directly since it has all the data from Cell 4
for code_name in sorted(full_results.keys()):
    code_results = full_results[code_name]
    code_type = code_results.get('type', 'Unknown')
    
    # Get code object for distance
    code = all_codes.get(code_name, (None, None))[1]
    d = code.metadata.get('distance', '?') if code else '?'
    
    # Get no-decode LER
    ler_no_decode = None
    for key, res in code_results.items():
        if isinstance(res, dict) and res.get('ler_no_decode') is not None:
            ler_no_decode = res['ler_no_decode']
            break
    
    nd_str = f"{ler_no_decode:.4f}" if ler_no_decode else 'N/A'
    row = f"{code_name:<35} | {code_type:<10} | {str(d):>2} | {nd_str:>10}"
    
    # Find best decoder
    best_ler = float('inf')
    best_decoder = None
    
    for dec_name in dec_names:
        res = code_results.get(dec_name, {})
        status = res.get('status') if isinstance(res, dict) else None
        ler = res.get('ler') if isinstance(res, dict) else None
        
        if status == 'SKIP':
            ler_str = 'SKIP'
        elif ler is not None:
            ler_str = f"{ler:.4f}"
            if ler < best_ler:
                best_ler = ler
                best_decoder = dec_name
        else:
            ler_str = 'FAIL'
        row += f" | {ler_str:>10}"
    
    best_str = best_decoder[:10] if best_decoder else 'N/A'
    row += f" | {best_str}"
    print(row)

print("-" * len(header))
print(f"\nTotal rows: {len(full_results)}")

LER COMPARISON TABLE (p=0.01) - ALL CODE TYPES

Total codes in all_codes: 46
Total codes in full_results: 46

Lower is better. Best decoder for each code highlighted.
SKIP = decoder incompatible (e.g., Chromobius requires color-code DEMs)

Code                                | Type       |  d |  No-decode | PyMatching | FusionBlos | BeliefMatc |      BPOSD |  Tesseract |  UnionFind |        MLE | Hypergraph | Chromobius | Best
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
BaconShor_3x3                       | Subsystem  |  3 |     0.0820 |     0.0160 |     0.0150 |     0.0180 |     0.0240 |     0.0160 |     0.0190 |     0.0190 |     0.1020 |       SKIP | FusionBlos
BalancedProduct_5x5_G1              | QLDPC      |  4 |     0.0240 |     0.0010 |     0.0040 |     0.0050 |     0.0050 |     0.0070 |     0.0030 |       FAIL |     0.0570 |       SK

In [7]:
# Cell 6: DECODER SPEED COMPARISON
from IPython.display import clear_output
clear_output(wait=True)

print("="*110)
print("DECODER SPEED COMPARISON (time in ms for 1000 shots)")
print("="*110)
print(f"\nTotal codes in full_results: {len(full_results)}\n")

# Header
dec_names = list(decoder_classes.keys())
header = f"{'Code':<30} {'Type':<10}"
for dec_name in dec_names:
    header += f" | {dec_name[:10]:>10}"
print(header)
print("-" * len(header))

# Use full_results directly since it has all the data from Cell 4
for code_name in sorted(full_results.keys()):
    code_results = full_results[code_name]
    code_type = code_results.get('type', 'Unknown')
    row = f"{code_name:<30} {code_type:<10}"
    
    for dec_name in dec_names:
        res = code_results.get(dec_name, {})
        time_ms = res.get('time_ms') if isinstance(res, dict) else None
        time_str = f"{time_ms:.1f}" if time_ms else 'FAIL'
        row += f" | {time_str:>10}"
    
    print(row)

print("-" * len(header))
print(f"\nTotal rows: {len(full_results)}")

DECODER SPEED COMPARISON (time in ms for 1000 shots)

Total codes in full_results: 46

Code                           Type       | PyMatching | FusionBlos | BeliefMatc |      BPOSD |  Tesseract |  UnionFind |        MLE | Hypergraph | Chromobius
--------------------------------------------------------------------------------------------------------------------------------------------------------------
BaconShor_3x3                  Subsystem  |        0.2 |        3.6 |       11.7 |       12.2 |       11.7 |        0.3 |        0.5 |        1.0 |       FAIL
BalancedProduct_5x5_G1         QLDPC      |        1.9 |       29.9 |      270.9 |      848.2 |       46.2 |        1.8 |       FAIL |        2.9 |       FAIL
BalancedProduct_7x7_G1         QLDPC      |        2.6 |       61.0 |      926.0 |     3528.1 |      332.7 |        2.4 |       FAIL |        4.0 |       FAIL
BareAncilla_713                Non-CSS    |        6.2 |        3.4 |        8.7 |       54.3 |       19.7 |        0.

In [8]:
# Cell 7: NOISE LEVEL SCALING TEST

print("="*110)
print("NOISE LEVEL SCALING TEST")
print("="*110)

# Pick a representative CSS code (RotatedSurface d=3) 
test_code_name = None
test_code_type = None
for name, (ctype, code) in all_codes.items():
    if 'RotatedSurface' in name and code.metadata.get('distance') == 3:
        test_code_name = name
        test_code_type = ctype
        break

if test_code_name is None:
    test_code_name = list(all_codes.keys())[0]
    test_code_type = all_codes[test_code_name][0]

test_code = all_codes[test_code_name][1]
print(f"\nTesting {test_code_name} ({test_code_type}) across noise levels with all decoders...\n")

noise_levels = [0.001, 0.005, 0.01, 0.02]

# Header
dec_names = list(decoder_classes.keys())
header = f"{'p':<10}"
for dec_name in dec_names:
    header += f" | {dec_name[:12]:>12}"
header += " | No-decode"
print(header)
print("-" * len(header))

for p in noise_levels:
    row = f"{p:<10.4f}"
    ler_no_decode = None
    
    for dec_name in dec_names:
        # test_decoder_on_code returns TestResult dataclass, not dict
        result = test_decoder_on_code(
            code=test_code,
            code_type=test_code_type,
            decoder_name=dec_name,
            decoder_class=decoder_classes[dec_name],
            p=p,
            shots=2000,
            rounds=3
        )
        
        # Access attributes directly (TestResult is a dataclass)
        if ler_no_decode is None and result.ler_no_decode is not None:
            ler_no_decode = result.ler_no_decode
        
        ler = result.ler
        ler_str = f"{ler:.6f}" if ler is not None else 'FAIL'
        row += f" | {ler_str:>12}"
    
    nd_str = f"{ler_no_decode:.6f}" if ler_no_decode else 'N/A'
    row += f" | {nd_str}"
    print(row)

print("-" * len(header))

NOISE LEVEL SCALING TEST

Testing RotatedSurface_[[9,1,3]] (CSS) across noise levels with all decoders...

p          |   PyMatching | FusionBlosso | BeliefMatchi |        BPOSD |    Tesseract |    UnionFind |          MLE |   Hypergraph |   Chromobius | No-decode
-------------------------------------------------------------------------------------------------------------------------------------------------------------


0.0010     |     0.000000 |     0.000000 |     0.001500 |     0.000500 |     0.000500 |     0.000000 |     0.000000 |     0.012000 |         FAIL | 0.011000
0.0050     |     0.005500 |     0.003500 |     0.008000 |     0.004000 |     0.003000 |     0.003500 |     0.007000 |     0.059500 |         FAIL | 0.052500
0.0050     |     0.005500 |     0.003500 |     0.008000 |     0.004000 |     0.003000 |     0.003500 |     0.007000 |     0.059500 |         FAIL | 0.052500
0.0100     |     0.020500 |     0.021000 |     0.015500 |     0.018000 |     0.015500 |     0.022000 |     0.019500 |     0.100000 |         FAIL | 0.092500
0.0100     |     0.020500 |     0.021000 |     0.015500 |     0.018000 |     0.015500 |     0.022000 |     0.019500 |     0.100000 |         FAIL | 0.092500
0.0200     |     0.059500 |     0.049000 |     0.056500 |     0.053500 |     0.052000 |     0.056500 |     0.076000 |     0.202000 |         FAIL | 0.177500
----------------------------------------------------------

In [9]:
# Cell 8: FINAL SUMMARY - ALL CODE TYPES

print("="*100)
print("FINAL DECODER SMOKE TEST SUMMARY - ALL CODE TYPES")
print("="*100)

# Count results by status
passed = 0
with_warnings = 0
skipped = 0
failed = 0
total_tests = 0

# Track by code type - include Color codes
results_by_type = {'CSS': {'pass': 0, 'warn': 0, 'skip': 0, 'fail': 0},
                   'Color': {'pass': 0, 'warn': 0, 'skip': 0, 'fail': 0},
                   'Non-CSS': {'pass': 0, 'warn': 0, 'skip': 0, 'fail': 0},
                   'Subsystem': {'pass': 0, 'warn': 0, 'skip': 0, 'fail': 0},
                   'Floquet': {'pass': 0, 'warn': 0, 'skip': 0, 'fail': 0},
                   'QLDPC': {'pass': 0, 'warn': 0, 'skip': 0, 'fail': 0}}

failed_tests = []

for code_name, code_results in full_results.items():
    code_type = code_results.get('type', 'Unknown')
    if code_type not in results_by_type:
        results_by_type[code_type] = {'pass': 0, 'warn': 0, 'skip': 0, 'fail': 0}
    
    for dec_name in decoder_classes.keys():
        res = code_results.get(dec_name, {})
        if isinstance(res, dict):
            total_tests += 1
            status = res.get('status')
            if status == 'OK':
                if res.get('warnings'):
                    with_warnings += 1
                    results_by_type[code_type]['warn'] += 1
                else:
                    passed += 1
                    results_by_type[code_type]['pass'] += 1
            elif status == 'WARN':
                with_warnings += 1
                results_by_type[code_type]['warn'] += 1
            elif status == 'SKIP':
                skipped += 1
                results_by_type[code_type]['skip'] += 1
            else:
                failed += 1
                results_by_type[code_type]['fail'] += 1
                failed_tests.append((code_name, dec_name, res.get('error', 'unknown')))

print(f"\n📊 OVERALL RESULTS:")
print(f"   Total decoder × code tests: {total_tests}")
print(f"   ✓ Passed:   {passed} ({100*passed/total_tests:.1f}%)")
print(f"   ⚠ Warnings: {with_warnings} ({100*with_warnings/total_tests:.1f}%)")
print(f"   - Skipped:  {skipped} ({100*skipped/total_tests:.1f}%) [expected - decoder incompatible]")
print(f"   ✗ Failed:   {failed} ({100*failed/total_tests:.1f}%)")

print(f"\n📊 RESULTS BY CODE TYPE:")
for ctype, counts in results_by_type.items():
    total = counts['pass'] + counts['warn'] + counts['skip'] + counts['fail']
    if total > 0:
        chromobius_note = " (Chromobius works!)" if ctype == 'Color' else ""
        print(f"   {ctype:>10}: {counts['pass']} pass, {counts['warn']} warn, {counts['skip']} skip, {counts['fail']} fail (total {total}){chromobius_note}")

print(f"\n🔧 DECODERS TESTED: {len(decoder_classes)}")
for dec_name in decoder_classes.keys():
    note = " (works on Color codes now!)" if dec_name == "Chromobius" else ""
    print(f"   - {dec_name}{note}")

print(f"\n📦 CODES TESTED BY TYPE:")
# Use categories from testing_utils (already loaded)
for code_type, codes_dict in categories.items():
    if codes_dict:
        print(f"\n   {code_type} ({len(codes_dict)} codes):")
        for name, code in list(codes_dict.items())[:10]:  # Limit to first 10
            d = code.metadata.get('distance', '?') if hasattr(code, 'metadata') else '?'
            print(f"     - {name}: [[{code.n},{code.k},{d}]]")
        if len(codes_dict) > 10:
            print(f"     ... and {len(codes_dict) - 10} more")

print("\n" + "="*100)
if failed == 0:
    print("✓ ALL TESTS PASSED (no unexpected failures)")
    if skipped > 0:
        print(f"  ({skipped} tests skipped - Chromobius skipped on non-color codes)")
else:
    print(f"⚠ {failed} TESTS FAILED (unexpected):")
    for code_name, dec_name, error in failed_tests[:20]:
        print(f"   - {code_name} × {dec_name}: {error}")
    if len(failed_tests) > 20:
        print(f"   ... and {len(failed_tests) - 20} more")
print("="*100)

FINAL DECODER SMOKE TEST SUMMARY - ALL CODE TYPES

📊 OVERALL RESULTS:
   Total decoder × code tests: 414
   ✓ Passed:   275 (66.4%)
   ⚠ Warnings: 72 (17.4%)
   - Skipped:  43 (10.4%) [expected - decoder incompatible]
   ✗ Failed:   24 (5.8%)

📊 RESULTS BY CODE TYPE:
          CSS: 160 pass, 20 warn, 24 skip, 12 fail (total 216)
        Color: 18 pass, 14 warn, 1 skip, 3 fail (total 36) (Chromobius works!)
      Non-CSS: 24 pass, 16 warn, 5 skip, 0 fail (total 45)
    Subsystem: 17 pass, 14 warn, 4 skip, 1 fail (total 36)
      Floquet: 8 pass, 0 warn, 1 skip, 0 fail (total 9)
        QLDPC: 48 pass, 8 warn, 8 skip, 8 fail (total 72)

🔧 DECODERS TESTED: 9
   - PyMatching
   - FusionBlossom
   - BeliefMatching
   - BPOSD
   - Tesseract
   - UnionFind
   - MLE
   - Hypergraph
   - Chromobius (works on Color codes now!)

📦 CODES TESTED BY TYPE:

   CSS (24 codes):
     - FourQubit422_[[4,2,2]]: [[4,2,2]]
     - C6: [[6,2,2]]
     - Steane_713: [[7,1,3]]
     - Shor_91: [[9,1,3]]
     - Re

In [10]:
# Cell 9: CHROMOBIUS DIRECT TEST ON COLOR CODES
# This cell tests Chromobius directly on color codes using ColorCodeMemoryExperiment
# which generates 4D detector coordinates with proper color annotations

import sys
# Force reimport to pick up latest code changes
modules_to_remove = [k for k in sys.modules.keys() if 'qectostim' in k]
for mod in modules_to_remove:
    del sys.modules[mod]

from qectostim.codes.color.triangular_colour import TriangularColourCode
from qectostim.codes.color.hexagonal_colour import HexagonalColourCode
from qectostim.codes.color.colour_code import ColourCode488
from qectostim.experiments.memory import ColorCodeMemoryExperiment
from qectostim.noise.models import CircuitDepolarizingNoise
import chromobius
import numpy as np

print("="*60)
print("CHROMOBIUS DIRECT TEST ON COLOR CODES")
print("="*60)
print("Uses ColorCodeMemoryExperiment for 4D detector coordinates")
print()

codes = [
    ('TriangularColour_d3', TriangularColourCode(distance=3)),
    ('HexagonalColour_d2', HexagonalColourCode(distance=2)),
    ('HexagonalColour_d3', HexagonalColourCode(distance=3)),
    ('Colour488_d3', ColourCode488(distance=3)),
]

shots = 1000
results = []

for name, code in codes:
    print(f"\n{'-'*50}")
    print(f"Testing: {name} (n={code.n}, k={code.k})")
    meta = code.metadata
    print(f"  stab_colors: {meta.get('stab_colors')}")
    print(f"  is_chromobius_compatible: {meta.get('is_chromobius_compatible')}")
    
    if not meta.get('is_chromobius_compatible', False):
        print(f"  ⚠️  SKIPPED - not 3-colorable (stabilizers form K4 complete graph)")
        results.append((name, 'SKIP', 'Not 3-colorable'))
        continue
    
    try:
        noise = CircuitDepolarizingNoise(p1=0.01, p2=0.01)
        exp = ColorCodeMemoryExperiment(code=code, rounds=3, noise_model=noise)
        circuit = noise.apply(exp.to_stim())
        
        # Sample from circuit directly
        sampler = circuit.compile_detector_sampler()
        dets, obs = sampler.sample(shots, separate_observables=True)
        
        # Compile Chromobius decoder
        dem = circuit.detector_error_model(decompose_errors=True)
        decoder = chromobius.compile_decoder_for_dem(dem)
        
        # Decode
        dets_packed = np.packbits(dets.astype(np.uint8), axis=1, bitorder='little')
        predictions = decoder.predict_obs_flips_from_dets_bit_packed(dets_packed)
        
        # Calculate error rate
        obs = obs.astype(np.uint8)
        errors = np.any(predictions != obs, axis=1).sum()
        ler = errors / shots
        print(f"  ✓ Chromobius decode: SUCCESS - LER = {ler:.4f} ({errors}/{shots} errors)")
        results.append((name, 'PASS', f'{ler:.4f}'))
        
    except Exception as e:
        print(f"  ✗ ERROR: {type(e).__name__}: {e}")
        results.append((name, 'FAIL', str(e)[:50]))

print("\n" + "="*60)
print("SUMMARY")
print("="*60)
for name, status, detail in results:
    emoji = {'PASS': '✓', 'SKIP': '⚠️', 'FAIL': '✗'}[status]
    print(f"  {emoji} {name}: {status} - {detail}")

CHROMOBIUS DIRECT TEST ON COLOR CODES
Uses ColorCodeMemoryExperiment for 4D detector coordinates


--------------------------------------------------
Testing: TriangularColour_d3 (n=7, k=1)
  stab_colors: [0, 1, 2]
  is_chromobius_compatible: True
  ✓ Chromobius decode: SUCCESS - LER = 0.0190 (19/1000 errors)

--------------------------------------------------
Testing: HexagonalColour_d2 (n=8, k=2)
  stab_colors: [0, 0, 1]
  is_chromobius_compatible: True
  ✓ Chromobius decode: SUCCESS - LER = 0.0190 (19/1000 errors)

--------------------------------------------------
Testing: HexagonalColour_d2 (n=8, k=2)
  stab_colors: [0, 0, 1]
  is_chromobius_compatible: True
  ✓ Chromobius decode: SUCCESS - LER = 0.0000 (0/1000 errors)

--------------------------------------------------
Testing: HexagonalColour_d3 (n=17, k=7)
  stab_colors: [0, 0, 0, 1, 1]
  is_chromobius_compatible: True
  ✓ Chromobius decode: SUCCESS - LER = 0.0000 (0/1000 errors)

-----------------------------------------------

In [11]:
# Debug: Try to instantiate the fracton codes directly
import sys
for mod in list(sys.modules.keys()):
    if 'qectostim' in mod:
        del sys.modules[mod]

print("="*70)
print("TRYING TO INSTANTIATE FRACTON CODES DIRECTLY")
print("="*70)

# Import topological module
from qectostim.codes import topological

# Try each fracton code
fracton_classes = ['HaahCode', 'XCubeCode', 'ChamonCode', 'CheckerboardCode', 
                    'FibonacciFractalCode', 'SierpinskiPrismCode']

for class_name in fracton_classes:
    cls = getattr(topological, class_name, None)
    if cls is None:
        print(f"❌ {class_name}: NOT FOUND")
        continue
    
    # Try different parameter combinations
    params_to_try = [
        {'L': 3},
        {'L': 4},
        {'L': 3, 'T': 2},
        {},
    ]
    
    for params in params_to_try:
        try:
            code = cls(**params) if params else cls()
            print(f"✓ {class_name}({params}): n={code.n}, k={code.k}")
            break
        except Exception as e:
            print(f"  {class_name}({params}): {type(e).__name__}: {str(e)[:50]}")

# Also try HoneycombCode
print("\n" + "="*70)
print("TRYING FLOQUET CODES")
print("="*70)

from qectostim.codes import floquet

for class_name in ['HoneycombCode', 'ISGFloquetCode']:
    cls = getattr(floquet, class_name, None)
    if cls is None:
        print(f"❌ {class_name}: NOT FOUND")
        continue
    
    params_to_try = [
        {'Lx': 2, 'Ly': 3},
        {'distance': 3},
        {'L': 3},
        {},
    ]
    
    for params in params_to_try:
        try:
            code = cls(**params) if params else cls()
            print(f"✓ {class_name}({params}): n={code.n}, k={code.k}")
            break
        except Exception as e:
            print(f"  {class_name}({params}): {type(e).__name__}: {str(e)[:60]}")

TRYING TO INSTANTIATE FRACTON CODES DIRECTLY
✓ HaahCode({'L': 3}): n=54, k=4
✓ XCubeCode({'L': 3}): n=81, k=15
✓ ChamonCode({'L': 3}): n=54, k=6
  CheckerboardCode({'L': 3}): ValueError: L must be even and >= 2
✓ CheckerboardCode({'L': 4}): n=64, k=10
  FibonacciFractalCode({'L': 3}): TypeError: FibonacciFractalCode.__init__() got an unexpected 
  FibonacciFractalCode({'L': 4}): TypeError: FibonacciFractalCode.__init__() got an unexpected 
  FibonacciFractalCode({'L': 3, 'T': 2}): TypeError: FibonacciFractalCode.__init__() got an unexpected 
✓ FibonacciFractalCode({}): n=23, k=3
  SierpinskiPrismCode({'L': 3}): TypeError: SierpinskiPrismCode.__init__() got an unexpected k
  SierpinskiPrismCode({'L': 4}): TypeError: SierpinskiPrismCode.__init__() got an unexpected k
  SierpinskiPrismCode({'L': 3, 'T': 2}): TypeError: SierpinskiPrismCode.__init__() got an unexpected k
✓ SierpinskiPrismCode({}): n=80, k=2

TRYING FLOQUET CODES
  HoneycombCode({'Lx': 2, 'Ly': 3}): TypeError: HoneycombCode.

In [12]:
# Cell: Verify fixes for k<=0 and d=? codes
import sys
# Force reimport
for mod in list(sys.modules.keys()):
    if 'qectostim' in mod:
        del sys.modules[mod]

from qectostim.codes import discover_all_codes

print("="*70)
print("VERIFYING k AND d FIXES")
print("="*70)

codes = discover_all_codes(max_qubits=100, timeout_per_code=3.0)

print(f"\nTotal codes discovered: {len(codes)}")

# Check for problematic codes
k_zero_codes = []
k_negative_codes = []
d_unknown_codes = []
all_good_codes = []

for name, code in codes.items():
    k = code.k
    d = code.metadata.get('distance', None)
    
    if k < 0:
        k_negative_codes.append((name, k, d))
    elif k == 0:
        k_zero_codes.append((name, k, d))
    elif d is None or d == '?':
        d_unknown_codes.append((name, k, d))
    else:
        all_good_codes.append((name, k, d))

print(f"\n✅ Codes with k>0 and known d: {len(all_good_codes)}")
print(f"⚠️  Codes with k=0: {len(k_zero_codes)}")
print(f"❌ Codes with k<0: {len(k_negative_codes)}")
print(f"⚠️  Codes with unknown d: {len(d_unknown_codes)}")

if k_negative_codes:
    print(f"\n❌ NEGATIVE K CODES (still broken):")
    for name, k, d in k_negative_codes:
        print(f"   {name}: k={k}, d={d}")

if k_zero_codes:
    print(f"\n⚠️  ZERO K CODES:")
    for name, k, d in k_zero_codes:
        print(f"   {name}: k={k}, d={d}")

if d_unknown_codes:
    print(f"\n⚠️  UNKNOWN DISTANCE CODES:")
    for name, k, d in d_unknown_codes[:10]:
        print(f"   {name}: k={k}, d={d}")
    if len(d_unknown_codes) > 10:
        print(f"   ... and {len(d_unknown_codes) - 10} more")

print("\n" + "="*70)
if len(k_negative_codes) == 0:
    print("✅ NO NEGATIVE K CODES - XCubeCode fix verified!")
else:
    print("❌ Still have negative k codes - needs investigation")

if len(k_zero_codes) <= 2:  # Some codes legitimately have k=0
    print("✅ ZERO K CODES MINIMIZED")
else:
    print(f"⚠️  Still have {len(k_zero_codes)} k=0 codes")

print("="*70)

VERIFYING k AND d FIXES



Total codes discovered: 43

✅ Codes with k>0 and known d: 39
⚠️  Codes with k=0: 0
❌ Codes with k<0: 0
⚠️  Codes with unknown d: 4

⚠️  UNKNOWN DISTANCE CODES:
   ProjectivePlaneSurface_[[13,1,None]]: k=1, d=None
   HGPHamming7_[[58,16,None]]: k=16, d=None
   CampbellDoubleHGP_3_[[13,1,None]]: k=1, d=None
   CampbellDoubleHGP_5_[[41,1,None]]: k=1, d=None

✅ NO NEGATIVE K CODES - XCubeCode fix verified!
✅ ZERO K CODES MINIMIZED
